# Weather Data from Insight

Insight measured the weather at Elysium Planitia.

## Preprocess Insight Weather Data

In [1]:
import pandas as pd
import json

In [2]:
import pathlib
from pathlib import Path

## Load Data

In [3]:
data_folder = Path("../dapi/insight-weather-cache")

data_file_paths = [p for p in data_folder.iterdir() if p.is_file() and p.suffix == ".json"]
data_file_paths.sort()

print(f"Number of json files {len(data_file_paths)}")
print(f"first path: {data_file_paths[:1]}")

Number of json files 907
first path: [PosixPath('../dapi/insight-weather-cache/data_1565210685.json')]


In [30]:
raw_data = {}

for f in data_file_paths:
    with open(f, "r") as fp:
        raw_data.update(json.load(fp))


raw_data = {k: v for k, v in raw_data.items() if k not in ["sol_keys", "validity_checks"]}

In [31]:
raw_data[list(raw_data.keys())[0]]

{'AT': {'av': -68.319, 'ct': 177556, 'mn': -97.958, 'mx': -25.718},
 'First_UTC': '2019-07-29T18:52:14Z',
 'HWS': {'av': 5.268, 'ct': 79473, 'mn': 0.18600000000000003, 'mx': 20.354},
 'Last_UTC': '2019-07-30T19:31:49Z',
 'PRE': {'av': 763.78, 'ct': 88778, 'mn': 743.3216, 'mx': 782.8978},
 'Season': 'winter',
 'WD': {'0': {'compass_degrees': 0.0,
   'compass_point': 'N',
   'compass_right': 0.0,
   'compass_up': 1.0,
   'ct': 151},
  '1': {'compass_degrees': 22.5,
   'compass_point': 'NNE',
   'compass_right': 0.382683432365,
   'compass_up': 0.923879532511,
   'ct': 147},
  '10': {'compass_degrees': 225.0,
   'compass_point': 'SW',
   'compass_right': -0.707106781187,
   'compass_up': -0.707106781187,
   'ct': 19045},
  '11': {'compass_degrees': 247.5,
   'compass_point': 'WSW',
   'compass_right': -0.923879532511,
   'compass_up': -0.382683432365,
   'ct': 4380},
  '12': {'compass_degrees': 270.0,
   'compass_point': 'W',
   'compass_right': -1.0,
   'compass_up': -0.0,
   'ct': 3134}

## Preprocessing Data

In [32]:
def parse_daily_data(daily_data, shared_keys, day, key_definition):

    print(f"parsing data for {day} (key = {key_definition['name']})")

    minimized_data = {
        "day": day
    }

    minimized_data.update(
        {
            k: daily_data[k]
            for k in [k["name"] for k in shared_keys]
        }
    )

    key_type = key_definition["type"]
    key_name = key_definition["name"]
    try:
        if key_type == "nest_order_1":
            values = daily_data.get(key_name)
            values = {
                f"{key_name}_{k}": v
                for k, v in values.items()
            }
        elif key_type == "nest_order_2":
            values = daily_data.get(key_name)
            values = {
                f"{k}_{v_k}": v_v
                for k, v in values.items()
                for v_k, v_v in v.items()
            }
        else:
            raise Exception(f"transformation for {key_type} is not yet implemented")

        minimized_data.update(values)
        return minimized_data
    except AttributeError as e:
        print(f"{day}: failed parsing values: {e}")
        return {}

    


In [33]:
raw_data.keys()

dict_keys(['239', '240', '241', '242', '243', '244', '245', '246', '247', '248', '249', '250', '251', '252', '253', '254', '255', '256', '257', '258', '259', '260', '261', '262', '263', '264', '265', '267', '268', '285', '286', '287', '288', '289', '290', '291', '292', '293', '294', '295', '296', '297', '298', '299', '300', '301', '302', '303', '304', '305', '306', '307', '308', '309', '310', '311', '312', '313', '314', '315', '316', '317', '318', '319', '320', '321', '322', '324', '325', '326', '327', '323', '328', '329', '330', '331', '332', '333', '334', '335', '336', '337', '338', '339', '340', '341', '342', '343', '344', '345', '346', '347', '348', '349', '350', '351', '352', '353', '354', '355', '356', '357', '358', '359', '360', '361', '362', '363', '364', '365', '366', '367', '368', '369', '370', '371', '372', '373', '374', '375', '376', '377', '378', '379', '380', '381', '382', '383', '384', '385', '386', '387', '388', '389', '390', '391', '392', '393', '394', '395', '396', '3

In [34]:
keys = [
    {
        "name": "First_UTC",
        "type": "flat"
    },
    {
        "name": "Last_UTC", 
        "type": "flat"
    },
    {
        "name": "Season",
        "type": "flat"
    },
    {
        "name": "AT",
        "type": "nest_order_1"
    },
    {
        "name": "HWS",
        "type": "nest_order_1"
    },
    {
        "name": "PRE",
        "type": "nest_order_1"
    },
    {
        "name": "WD",
        "type": "nest_order_2"
    }
]

shared_keys = [
    k for k in keys
    if k["type"] == "flat"
]


In [35]:
parse_daily_data(raw_data["239"], shared_keys, "239", {
    "name": "AT",
    "type": "nest_order_1"
})

parsing data for 239 (key = AT)


{'day': '239',
 'First_UTC': '2019-07-29T18:52:14Z',
 'Last_UTC': '2019-07-30T19:31:49Z',
 'Season': 'winter',
 'AT_av': -68.319,
 'AT_ct': 177556,
 'AT_mn': -97.958,
 'AT_mx': -25.718}

In [39]:

data = {}

for k in keys:
    if k["type"] != "flat":
        data[k["name"]] = pd.DataFrame(
            [
                parse_daily_data(raw_data[day], shared_keys, day, k)
                for day in raw_data
            ]
        )

parsing data for 356 (key = PRE)
parsing data for 357 (key = PRE)
parsing data for 358 (key = PRE)
parsing data for 359 (key = PRE)
parsing data for 360 (key = PRE)
parsing data for 361 (key = PRE)
parsing data for 362 (key = PRE)
parsing data for 363 (key = PRE)
parsing data for 364 (key = PRE)
parsing data for 365 (key = PRE)
parsing data for 366 (key = PRE)
parsing data for 367 (key = PRE)
parsing data for 368 (key = PRE)
parsing data for 369 (key = PRE)
parsing data for 370 (key = PRE)
parsing data for 371 (key = PRE)
parsing data for 372 (key = PRE)
parsing data for 373 (key = PRE)
parsing data for 374 (key = PRE)
parsing data for 375 (key = PRE)
parsing data for 376 (key = PRE)
parsing data for 377 (key = PRE)
parsing data for 378 (key = PRE)
parsing data for 379 (key = PRE)
parsing data for 380 (key = PRE)
parsing data for 381 (key = PRE)
parsing data for 382 (key = PRE)
parsing data for 383 (key = PRE)
parsing data for 384 (key = PRE)
parsing data for 385 (key = PRE)
parsing da

In [41]:
data.keys()

dict_keys(['AT', 'HWS', 'PRE', 'WD'])

In [42]:
data["AT"]

,day,First_UTC,Last_UTC,Season,AT_av,AT_ct,AT_mn,AT_mx
0,239,2019-07-29T18:52:14Z,2019-07-30T19:31:49Z,winter,-68.319,177556.0,-97.958,-25.718
1,240,2019-07-30T19:31:50Z,2019-07-31T20:11:24Z,winter,-68.593,177554.0,-98.453,-25.049
2,241,2019-07-31T20:11:25Z,2019-08-01T20:50:59Z,winter,-68.478,172434.0,-100.484,-26.237
3,242,2019-08-01T20:51:00Z,2019-08-02T21:30:35Z,winter,-69.130,177556.0,-99.099,-26.090
4,243,2019-08-02T21:30:36Z,2019-08-03T22:10:10Z,winter,-69.303,177553.0,-99.819,-25.934
...,...,...,...,...,...,...,...,...
494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
495,1081,2021-12-10T23:22:46Z,2021-12-11T23:04:24Z,spring,-55.400,267798.0,-118.891,-13.051
496,1086,2021-12-16T01:42:46Z,2021-12-17T02:22:20Z,spring,-61.626,177556.0,-95.073,-10.217
497,1087,2021-12-17T02:22:21Z,2021-12-17T19:33:42Z,spring,-55.617,123714.0,-94.048,-12.977


- AT Object; per-Sol atmospheric temperature sensor summary data
- HWS Object; per-Sol horizontal wind speed sensor summary data
- PRE Object; per-Sol atmospheric pressure sensor summary data
- WD Object; per-Sol wind direction sensor summary data
- Season String; per-Sol season on Mars; one of [“winter”, “spring”, “summer”, “fall”]
- First_UTC Time of first datum, of any sensor, for the Sol (UTC; YYYY-MM-DDTHH:MM:SSZ)
- Last_UTC Time of last datum, of any sensor, for the Sol (UTC; YYYY-MM-DDTHH:MM:SSZ)
- av Average of samples over the Sol (°F for AT; m/s for HWS; Pa for PRE)
- ct Total number of recorded samples over the Sol
- mn Minimum data sample over the sol (same units as av)
- mx Maximum data sample over the sol (same units as av)

## AT: Atmospheric Temperature

In [61]:
df_at = data["AT"].copy()

In [62]:
df_at.head()

,day,First_UTC,Last_UTC,Season,AT_av,AT_ct,AT_mn,AT_mx
0,239,2019-07-29T18:52:14Z,2019-07-30T19:31:49Z,winter,-68.319,177556.0,-97.958,-25.718
1,240,2019-07-30T19:31:50Z,2019-07-31T20:11:24Z,winter,-68.593,177554.0,-98.453,-25.049
2,241,2019-07-31T20:11:25Z,2019-08-01T20:50:59Z,winter,-68.478,172434.0,-100.484,-26.237
3,242,2019-08-01T20:51:00Z,2019-08-02T21:30:35Z,winter,-69.130,177556.0,-99.099,-26.090
4,243,2019-08-02T21:30:36Z,2019-08-03T22:10:10Z,winter,-69.303,177553.0,-99.819,-25.934


Average atomospheric temperature (Degrees Celcius) for each each sol.

In [95]:
_deepnote_run_altair(df_at, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"point","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"day","type":"ordinal","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"AT_av","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"color":{"field":"Season","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

## HWS: Horizontal Wind Speed

In [63]:
df_hws = data["HWS"].copy()

In [64]:
df_hws

,day,First_UTC,Last_UTC,Season,HWS_av,HWS_ct,HWS_mn,HWS_mx
0,239,2019-07-29T18:52:14Z,2019-07-30T19:31:49Z,winter,5.268,79473.0,0.186,20.354
1,240,2019-07-30T19:31:50Z,2019-07-31T20:11:24Z,winter,4.532,77229.0,0.168,15.794
2,241,2019-07-31T20:11:25Z,2019-08-01T20:50:59Z,winter,4.571,72470.0,0.126,16.189
3,242,2019-08-01T20:51:00Z,2019-08-02T21:30:35Z,winter,4.681,76312.0,0.162,17.065
4,243,2019-08-02T21:30:36Z,2019-08-03T22:10:10Z,winter,5.205,80811.0,0.158,20.541
...,...,...,...,...,...,...,...,...
494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
495,1081,2021-12-10T23:22:46Z,2021-12-11T23:04:24Z,spring,5.231,118042.0,0.283,17.399
496,1086,2021-12-16T01:42:46Z,2021-12-17T02:22:20Z,spring,4.309,78906.0,0.276,20.544
497,1087,2021-12-17T02:22:21Z,2021-12-17T19:33:42Z,spring,5.331,56960.0,0.307,19.441


Average horizontal wind speed (m/s)

In [70]:
_deepnote_run_altair(df_hws, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"point","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"day","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"HWS_av","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"color":{"field":"Season","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

## PRE: Pressure

In [71]:
df_pre = data["PRE"].copy()

In [73]:
df_pre.head()

,day,First_UTC,Last_UTC,Season,PRE_av,PRE_ct,PRE_mn,PRE_mx
0,239,2019-07-29T18:52:14Z,2019-07-30T19:31:49Z,winter,763.780,88778.0,743.3216,782.8978
1,240,2019-07-30T19:31:50Z,2019-07-31T20:11:24Z,winter,763.364,88776.0,743.5089,781.8654
2,241,2019-07-31T20:11:25Z,2019-08-01T20:50:59Z,winter,763.512,86817.0,743.8084,782.3278
3,242,2019-08-01T20:51:00Z,2019-08-02T21:30:35Z,winter,763.203,88779.0,743.8948,781.4774
4,243,2019-08-02T21:30:36Z,2019-08-03T22:10:10Z,winter,763.212,88776.0,743.4844,782.0221


Average pressure (Pa)

In [78]:
_deepnote_run_altair(df_pre, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"point","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"day","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"PRE_av","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"color":{"field":"Season","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

## WD: wind speed

- compass_degrees Number; the compass direction of the center of the compass point; degrees
  - N.B. the wind is blowing from this direction ±11.25°
- compass_point String; the name of the compass point e.g. “N” for North, or “ESE” for East-SouthEast
- compass_right Number; the positive-right (positive-east), horizontal component of a unit vector indicating - the direction of the compass point
- compass_up Number; the positive--up (positive-north), vertical component of a unit vector indicating the - direction of the compass point
- ct Number; the number of samples for the Sol in the 11.25° around this compass point

In [79]:
df_wd = data["WD"].copy()

In [81]:
df_wd.head()

,day,First_UTC,Last_UTC,Season,0_compass_degrees,0_compass_point,0_compass_right,0_compass_up,0_ct,1_compass_degrees,...,9_compass_degrees,9_compass_point,9_compass_right,9_compass_up,9_ct,most_common_compass_degrees,most_common_compass_point,most_common_compass_right,most_common_compass_up,most_common_ct
0,239,2019-07-29T18:52:14Z,2019-07-30T19:31:49Z,winter,0.0,N,0.0,1.0,151.0,22.5,...,202.5,SSW,-0.382683,-0.92388,12671.0,225.0,SW,-0.707107,-0.707107,19045.0
1,240,2019-07-30T19:31:50Z,2019-07-31T20:11:24Z,winter,0.0,N,0.0,1.0,1.0,22.5,...,202.5,SSW,-0.382683,-0.92388,12302.0,225.0,SW,-0.707107,-0.707107,18010.0
2,241,2019-07-31T20:11:25Z,2019-08-01T20:50:59Z,winter,NaN,NaN,NaN,NaN,NaN,22.5,...,202.5,SSW,-0.382683,-0.92388,10712.0,157.5,SSE,0.382683,-0.923880,15912.0
3,242,2019-08-01T20:51:00Z,2019-08-02T21:30:35Z,winter,0.0,N,0.0,1.0,7.0,22.5,...,202.5,SSW,-0.382683,-0.92388,10166.0,225.0,SW,-0.707107,-0.707107,17251.0
4,243,2019-08-02T21:30:36Z,2019-08-03T22:10:10Z,winter,0.0,N,0.0,1.0,113.0,22.5,...,202.5,SSW,-0.382683,-0.92388,14228.0,135.0,SE,0.707107,-0.707107,14697.0


Most common wind direction in degrees for each sol.

In [89]:
_deepnote_run_altair(df_wd, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"point","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"day","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"most_common_compass_degrees","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"color":{"field":"Season","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3bedb077-3beb-4d95-9880-c580ff14710c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>